# PDF extraction with cleaning

In [1]:
import pdfplumber
import pandas as pd
import re,csv
import numpy as np

#arguments that will read entire narration
def tb():
    ts={
         "vertical_strategy":"lines",
#          "horizontal_strategy":"lines",
        "horizontal_strategy":"text",
         "min_words_horizontal":1  
    }
    return ts

#PDF extraction code
file = 'ICICI.pdf'
pdf = pdfplumber.open(file)
pdf_text=[]
counter = 0
for page in pdf.pages:
    page = pdf.pages[counter] 
    pdfdata = page.extract_table(tb())
    dff = pd.DataFrame(pdfdata)
    pdf_text.append(dff)
    counter = counter + 1
pdf_text
d1 = pd.concat(pdf_text, axis=0)
d1 = d1.replace('',np.nan,regex=True)
d1 = d1.replace(r'\n',' ', regex=True)
d =d1
df1 = d.rename(columns = {0:"Date"})


#Code of cleaning 
for i in df1.columns:
    if 'date' in str(i).lower():
        dat=str(i)
        df1=df1.rename(columns = {dat:'Date'})
df1 = df1.loc[:,~df1.columns.duplicated()] # remove duplicate columns
# df1=df1.dropna(how='all',axis=1)
df2 = pd.DataFrame()                       
counter=0
blank_df=pd.DataFrame()


# narration df1shift with respect to date
def rolling_group(val):
    if pd.notnull(val): rolling_group.group +=1
    return rolling_group.group
rolling_group.group = 0
def joinFunc(g,column):
    col =g[column]
    joiner = "/" if column == "Date" else ""
    s = joiner.join([str(each) for each in col if pd.notnull(each)])
    s = re.sub("(?<=&)"+joiner," ",s)
    s = re.sub("(?<=-)"+joiner,"",s)
    s = re.sub(joiner*2,joiner,s)
    
    return s
groups = df1.groupby(df1['Date'].apply(rolling_group),as_index=False)
groupFunct = lambda g: pd.Series([joinFunc(g,col) for col in g.columns],index=g.columns)
x=groups.apply(groupFunct)
x['Date'] = pd.to_datetime(x['Date'],errors='coerce',dayfirst=True).dt.strftime('%Y-%m-%d')
x

try:
#delete rows which have opening balance etc.
    x = x[~x['Particulars'].isin(['Opening Balance','Balance B/F','C/F','B/F','Total'])]
    x = x[~x['Particulars'].str.contains('Opening|Closing|Brought|Carried',flags=re.IGNORECASE)]
    print("Particulars")
except:
    pass


# data shift to our custom based headers
list=['Date','Narration','Debit','Credit','Type','df','dfd']
for col in x.columns:
    x=x.replace('',np.nan)

    x[col] = x[col].replace(np.nan,0)
    x[col] = x[col].replace('NaN',0)
    x[col] = x[col].astype(str)

    if any(x[col].iloc[:5].str.contains(r'\d{4}-\d{2}-\d{2}|d{2}-\d{2}-\d{4}', regex=True, na=True)):
      
        df2.insert(counter,list[counter],x[col])
        counter+=1
        
    # To bundle up the Narration
    elif any(x[col].str.len()>25):
        x[col] = x[col].replace('nan','0')
        x[col] = x[col].replace('-','0')
        df2.insert(counter,list[counter],x[col])
        counter+=1
    
    elif any(x[col].iloc[:5].str.contains(r'\d+\.\d+', regex=True, na=True)):
        x[col] = x[col].replace('nan','0')
        x[col] = x[col].replace('-','0')
        df2.insert(counter,list[counter],x[col])
        counter+=1
        

    elif all(x[col].iloc[:5].str.contains(r'Cr|Dr|CR|DR|Credit|Debit|CREDIT|DEBIT',regex=True, na=True)):

        df2['Amount'] = df2.Debit.str.cat(x[col])
try:
#replace nan to 0 value in credit and debit
    df2 = df2.loc[~((df2['Credit'] == '0') & (df2['Debit'] == '0') & (df2['Date'] == '0'))]
    df2 = df2.fillna({'Credit':0.00,'Debit':0.00})
    df2=df2[(df2 == 0.0).sum(1) <= 2]
except:
    pass

try:
    df2['Credit']=df2['Credit'].replace('-',0.00)
    df2['Debit']=df2['Debit'].replace('-',0.00)
    df2=df2[(df2 == 0.0).sum(1) <= 2]
except:
    pass
try:
    df2['Credit'] = df2['Credit'].str.extract(r'(\d+.\d+)').astype('float')
    df2['Debit'] = df2['Debit'].str.extract(r'(\d+.\d+)').astype('float')
except:
    pass
df2 = df2.fillna({'Credit':0,'Debit':0})


df2=df2[['Date','Narration','Debit','Credit']]
indexAge = df2[(df2['Date'] == '0')].index
df2.drop(indexAge , inplace=True)
df2.head(20)

C:\Users\te\anaconda3\lib\site-packages\pandas\core\indexes\extension.py:101: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  return Index(result, name=self.name)


,Date,Narration,Debit,Credit
0,0.0,Transaction Date05/04/2022,Transaction RemarksCMS/ 140791982/ICICI PRUDEN...,0.0
1,0.0,12/04/2022,LIFE INSURANCE COCMS/ 140901236/ICICI PRUDENTIAL,0.0
2,0.0,26/04/2022,LIFE INSURANCE COCMS/ 141062361/ICICI PRUDENTIAL,0.0
3,0.0,04/05/2022,LIFE INSURANCE COCMS/ 141131654/ICICI PRUDENTIAL,0.0
4,0.0,10/05/2022,LIFE INSURANCE COCMS/ 141214230/ICICI PRUDENTIAL,0.0
5,0.0,24/05/2022,LIFE INSURANCE COCMS/ 141349520/ICICI PRUDENTIAL,0.0
6,0.0,07/06/2022,LIFE INSURANCE COCMS/ 141491485/ICICI PRUDENTIAL,0.0
7,0.0,28/06/2022,LIFE INSURANCE COCMS/ 141720412/ICICI PRUDENTIAL,0.0
8,0.0,30/06/2022,LIFE INSURANCE CO002401510189:Int.Pd:30-03-202...,0.0
9,0.0,12/07/2022,29-06-2022CMS/ 141870497/ICICI PRUDENTIAL,0.0


In [16]:
import pdfplumber
import pandas as pd
import re,csv
import numpy as np

#arguments that will read entire narration
def tb():
    ts={      
         "horizontal_strategy":"text"
    }
    return ts

#PDF extraction code
file = r'ICICI.pdf'

pdf = pdfplumber.open(file)
pdf_text=[]
pdf_text1=[]
counter = 0
for page in pdf.pages:
    page = pdf.pages[counter] 
    pdfdata = page.extract_table()
    dff = pd.DataFrame(pdfdata)
#     print(dff)
    
    try:
        for i in dff.iloc[0]:
            if 'Date' in str(i):
                pdfdata = page.extract_table(tb())
                dff = pd.DataFrame(pdfdata)
                pdf_text.append(dff) 
                
    except:
        pass
    counter = counter + 1
pdf_text
try:
    d1 = pd.concat(pdf_text, axis=0)
except:
    pass
try:
    d1 = pd.concat(pdf_text1, axis=0)
except:
    pass
d1 = d1.replace('',np.nan,regex=True)
d1 = d1.replace(r'\n',' ', regex=True)
d =d1
df1 = d.rename(columns = {0:"Date"})

# df1
# Code of cleaning 
for i in df1.columns:
    if 'date' in str(i).lower():
        dat=str(i)
        df1=df1.rename(columns = {dat:'Date'})
df1 = df1.loc[:,~df1.columns.duplicated()] # remove duplicate columns
# df1=df1.dropna(how='all',axis=1)
df2 = pd.DataFrame()                       
counter=0
blank_df=pd.DataFrame()
df1


#narration shift with respect to date
def rolling_group(val):
    if pd.notnull(val): rolling_group.group +=1
    return rolling_group.group
rolling_group.group = 0
def joinFunc(g,column):
    col =g[column]
    joiner = "/" if column == "Date" else ""
    s = joiner.join([str(each) for each in col if pd.notnull(each)])
    s = re.sub("(?<=&)"+joiner," ",s)
    s = re.sub("(?<=-)"+joiner,"",s)
    s = re.sub(joiner*2,joiner,s)
    
    return s
groups = df1.groupby(df1['Date'].apply(rolling_group),as_index=False)
groupFunct = lambda g: pd.Series([joinFunc(g,col) for col in g.columns],index=g.columns)
x=groups.apply(groupFunct)
x['Date'] = pd.to_datetime(x['Date'],errors='coerce',dayfirst=True).dt.strftime('%Y-%m-%d')

try:
#delete rows which have opening balance etc.
    x = x[~x['Particulars'].isin(['Opening Balance','Balance B/F','C/F','B/F','Total'])]
    x = x[~x['Particulars'].str.contains('Opening|Closing|Brought|Carried',flags=re.IGNORECASE)]
#     print("Particulars")
except:
    pass


# data shift to our custom based headers
list=['Date','Narration','Debit','Credit','Type','df','dfd','dfdf']
for col in x.columns:
    x=x.replace('',np.nan)

    x[col] = x[col].replace(np.nan,0)
    x[col] = x[col].replace('NaN',0)
    x[col] = x[col].astype(str)

    if any(x[col].iloc[:5].str.contains(r'\d{4}-\d{2}-\d{2}|d{2}-\d{2}-\d{4}', regex=True, na=True)):
      
        df2.insert(counter,list[counter],x[col])
        counter+=1
        
    # To bundle up the Narration
    elif any(x[col].str.len()>25):
        x[col] = x[col].replace('nan','0')
        x[col] = x[col].replace('-','0')
        df2.insert(counter,list[counter],x[col])
        counter+=1
    
    elif any(x[col].iloc[:5].str.contains(r'\d+\.\d+', regex=True, na=True)):
        x[col] = x[col].replace('nan','0')
        x[col] = x[col].replace('-','0')
        df2.insert(counter,list[counter],x[col])
        counter+=1
        

    elif all(x[col].iloc[:5].str.contains(r'Cr|Dr|CR|DR|Credit|Debit|CREDIT|DEBIT',regex=True, na=True)):

        df2['Amount'] = df2.Debit.str.cat(x[col])
try:
#replace nan to 0 value in credit and debit
    df2 = df2.loc[~((df2['Credit'] == '0') & (df2['Debit'] == '0') & (df2['Date'] == '0'))]
    df2 = df2.fillna({'Credit':0.00,'Debit':0.00})
    df2=df2[(df2 == 0.0).sum(1) <= 2]
except:
    pass
df2
# try:
#     df2['Credit']=df2['Credit'].replace('-',0.00)
#     df2['Debit']=df2['Debit'].replace('-',0.00)
#     df2=df2[(df2 == 0.0).sum(1) <= 2]
# except:
#     pass
# try:
#     df2['Credit'] = df2['Credit'].str.extract(r'(\d+.\d+)').astype('float')
#     df2['Debit'] = df2['Debit'].str.extract(r'(\d+.\d+)').astype('float')
# except:
#     pass
# df2 = df2.fillna({'Credit':0,'Debit':0})


# df2=df2[['Date','Narration','Debit','Credit']]
# indexAge = df2[(df2['Date'] == '0')].index
# df2.drop(indexAge , inplace=True)
# df2

C:\Users\te\anaconda3\lib\site-packages\pandas\core\indexes\extension.py:101: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  return Index(result, name=self.name)


,Date,Narration,Debit,Credit,Type,df
0,0.0,Transaction Date05/04/2022,Transaction RemarksCMS/ 140791982/ICICI PRUDEN...,Withdrawal Amount(INR )0.0,Deposit Amount(INR )735.00,Balance (INR )87958.75
1,0.0,12/04/2022,LIFE INSURANCE COCMS/ 140901236/ICICI PRUDENTIAL,0.0,764.00,88722.75
2,0.0,26/04/2022,LIFE INSURANCE COCMS/ 141062361/ICICI PRUDENTIAL,0.0,33.00,88755.75
3,0.0,04/05/2022,LIFE INSURANCE COCMS/ 141131654/ICICI PRUDENTIAL,0.0,410.00,89165.75
4,0.0,10/05/2022,LIFE INSURANCE COCMS/ 141214230/ICICI PRUDENTIAL,0.0,98.00,89263.75
5,0.0,24/05/2022,LIFE INSURANCE COCMS/ 141349520/ICICI PRUDENTIAL,0.0,33.00,89296.75
6,0.0,07/06/2022,LIFE INSURANCE COCMS/ 141491485/ICICI PRUDENTIAL,0.0,508.00,89804.75
7,0.0,28/06/2022,LIFE INSURANCE COCMS/ 141720412/ICICI PRUDENTIAL,0.0,33.00,89837.75
8,0.0,30/06/2022,LIFE INSURANCE CO002401510189:Int.Pd:30-03-202...,0.0,673.00,90510.75
9,0.0,12/07/2022,29-06-2022CMS/ 141870497/ICICI PRUDENTIAL,0.0,508.00,91018.75
